### 📥 Leitura da Base de Treinamento

Este trecho localiza automaticamente a raiz do projeto e carrega o arquivo df_trein.parquet, salvo na pasta data/processed.

Após validar a existência do arquivo, o dataset é lido com pandas e são exibidos o shape e os tipos das colunas, garantindo que a base de treinamento está correta antes da modelagem.


In [1]:
import pandas as pd
from pathlib import Path

# Descobre a raiz do projeto (assume que o notebook está em notbooks/)
try:
    ROOT = Path(__file__).resolve().parents[1]
except NameError:
    # __file__ não existe em notebooks; usa o cwd como base
    ROOT = Path.cwd().resolve().parent

data_path = ROOT / 'data' / 'processed' / 'df_trein.parquet'

if not data_path.exists():
    raise FileNotFoundError(f"Arquivo não encontrado: {data_path}")

df = pd.read_parquet(data_path)

print("Shape do dataset:", df.shape)
print("\nTipos de dados:\n")
print(df.dtypes)

Shape do dataset: (1390, 19)

Tipos de dados:

ANO                   int64
IDADE                 int64
FASE                  int64
DEFASAGEM             int64
IAA                 float64
IEG                 float64
IDA                 float64
IAN                 float64
IPS                 float64
IPV                 float64
NOTA_MAT            float64
NOTA_POR            float64
ABANDONO              int64
IDA_MISSING           int64
IEG_MISSING           int64
IPV_MISSING           int64
IPS_MISSING           int64
NOTA_MAT_MISSING      int64
NOTA_POR_MISSING      int64
dtype: object


### 🤖 Treinamento e Avaliação do Modelo (2022 → 2023)

Este bloco treina um GradientBoostingClassifier para prever o target ABANDONO, usando uma validação temporal: treino com dados de 2022 e teste com dados de 2023 (evita vazamento e simula uso em produção).

Etapas principais:

 - Sanity checks: garante que o target existe e que o dataset contém apenas os anos esperados (2022 e 2023).

 - Separação de features/target e divisão temporal (train=2022, test=2023).

 - Treinamento com pesos (sample_weight) para dar mais importância à classe 1 (abandono), lidando com desbalanceamento.

 - Avaliação por probabilidades com métricas mais adequadas para classificação desbalanceada:

 - ROC-AUC (separação geral)

 - PR-AUC (foco na classe positiva)

 - Ajuste de threshold (apenas diagnóstico): escolhe o limiar que maximiza F1 no conjunto de teste e imprime matriz de confusão e relatório de classificação.


In [2]:
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_curve, confusion_matrix, classification_report

TARGET = "ABANDONO"

# ---- sanity checks ----
assert TARGET in df.columns, f"Target {TARGET} não está no dataset"
assert set(df["ANO"].unique()) == {2022, 2023}, "Esperado ANO em {2022, 2023}"

features = [c for c in df.columns if c != TARGET]

train = df[df["ANO"] == 2022].copy()
test  = df[df["ANO"] == 2023].copy()

# treino 2022 / teste 2023
X_train, y_train = train[features], train[TARGET]
X_test,  y_test  = test[features],  test[TARGET]

gb = GradientBoostingClassifier(
    random_state=42,
    n_estimators=300,
    learning_rate=0.05,
    max_depth=3,         # profundidade das árvores base
    subsample=0.8
)

# sample_weight para dar mais peso à classe 1 (abandono)
w_pos = 2.5
sample_weight = np.where(y_train == 1, w_pos, 1.0)

gb.fit(X_train, y_train, sample_weight=sample_weight)

proba = gb.predict_proba(X_test)[:, 1]

print("ROC-AUC:", roc_auc_score(y_test, proba))
print("PR-AUC :", average_precision_score(y_test, proba))

# threshold por F1 (diagnóstico)
prec, rec, thr = precision_recall_curve(y_test, proba)
f1 = 2 * prec * rec / (prec + rec + 1e-12)
best_idx = f1.argmax()
best_thr = thr[best_idx - 1] if best_idx > 0 else 0.5
print("Best thr:", best_thr, "F1:", f1[best_idx], "P:", prec[best_idx], "R:", rec[best_idx])

pred = (proba >= best_thr).astype(int)
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred, digits=3))

ROC-AUC: 0.635028395456727
PR-AUC : 0.38804779917600796
Best thr: 0.10869081692122601 F1: 0.4550561797748922 P: 0.30916030534351147 R: 0.8617021276595744
[[169 363]
 [ 26 162]]
              precision    recall  f1-score   support

           0      0.867     0.318     0.465       532
           1      0.309     0.862     0.454       188

    accuracy                          0.460       720
   macro avg      0.588     0.590     0.460       720
weighted avg      0.721     0.460     0.462       720

